In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./raw_data/NWM_data_updated_land3.xlsx", engine='openpyxl')
a=xl.sheet_names


sheet_no = list(range(2, 32))

In [2]:
#loading the data set
dataframe_collection = {}
i=0
for i in range(33):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])


dataframe_collection[0]=dataframe_collection[0].dropna().reset_index(drop=True)

In [3]:
dataframe_collection[0]

,STAID_1,Dist2Sea(degree),LandUse,SoilTexture-1layer,SoilTexture-2layer,SoilTexture-3layer,SoilTexture-4layer,SoilTexture-5layer,SoilTexture-6layer,SoilTexture-7layer,SoilTexture-8layer,SoilTexture-9layer,SoilTexture-10layer,TopographyIndex,DrainageArea(sqml)
0,02217297,3.947508,3.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.340611,398.00
1,02217475,3.946926,3.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.444433,2950.00
2,02217615,4.119859,3.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.210577,3.05
3,02217643,4.080818,3.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.590834,275.00
4,02217500,3.880879,3.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.480809,99.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,02375000,0.431609,3.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,15.0,8.489074,394.00
278,03500000,5.079052,3.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,3.0,15.0,6.324007,436.00
279,03505550,5.207637,3.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,3.0,15.0,6.010764,3.60
280,03510577,5.322825,3.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,3.0,15.0,6.104362,352.00


In [4]:

x_mean = pd.read_csv("./raw_data/x_mean.csv", sep='\t')
x_std =  pd.read_csv("./raw_data/x_std.csv", sep='\t')
y_mean =  pd.read_csv("./raw_data/y_mean.csv", sep='\t')
y_std =  pd.read_csv("./raw_data/y_std.csv", sep='\t')

In [5]:
from keras.models import load_model
import statistics
import math
pred = [None] * 30
std = [None] * 30
std_err = [None] *30
obsflow = dataframe_collection[2]
for l in range(0,30) :
     
    model = load_model('./temporal_DLmodels/my_model_%d_day.h5'%(l))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    modelflow= dataframe_collection[l+3]
    temp =  [0] * 282
    for index, row in dataframe_collection[0].iterrows():
        column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
        df = pd.DataFrame(columns = column_names)  
        k=0
        d=0
      
        sta_id = dataframe_collection[0].iloc[index,0]
        dis2sea=dataframe_collection[0].iloc[index,1]
        land_use=dataframe_collection[0].iloc[index,2]
        soil_texture1=dataframe_collection[0].iloc[index,3]
        soil_texture2=dataframe_collection[0].iloc[index,4]
        soil_texture3=dataframe_collection[0].iloc[index,5]
        soil_texture4=dataframe_collection[0].iloc[index,6]
        soil_texture5=dataframe_collection[0].iloc[index,7]
        soil_texture6=dataframe_collection[0].iloc[index,8]
        soil_texture7=dataframe_collection[0].iloc[index,9]
        soil_texture8=dataframe_collection[0].iloc[index,10]
        soil_texture9=dataframe_collection[0].iloc[index,11]
        soil_texture10=dataframe_collection[0].iloc[index,12]
        topoindex = dataframe_collection[0].iloc[index,13]
        drainagearea =  dataframe_collection[0].iloc[index,14]
        obs = obsflow[sta_id]/drainagearea
        mod = modelflow[sta_id]/drainagearea

        for i in range(481,979):
            df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
            k=k+1
        df =df.dropna()
            
        if df.empty:
            temp[index] = 0
            d=d+1
            print(index)
        else :
            x, y = df.iloc[ :, 0:15], df.iloc[:,15]
            x-= x_mean.iloc[l,1:16]
            x /= x_std.iloc[l,1:16]
            
            z=model.predict(x)
            
            x=x*x_std.iloc[l,1:16]
            x=x +x_mean.iloc[l,1:16]
            
            z-= y_mean.iloc[l,1]
            z /= y_std.iloc[l,1]
            
            z=z.ravel()
            z=abs(z)
            a=x.iloc[:,14]
            b=a-(1*z)
            c=a+(z*1)
            res=y.between(b,c)
            count = np.count_nonzero(res)
            temp[index] = count/len(y)   
       
    std[l]= statistics.stdev(temp)    
    pred[l] = sum(temp)/(282-d) 
    std_err[l] = (std[l]*2)/math.sqrt(282-d)
    del model


97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251
97
98
197
251


In [6]:
d_x= pd.DataFrame(pred)
d_y=pd.DataFrame(std)
d_z=pd.DataFrame(std_err)
d_xy=pd.concat([d_x, d_y, d_z], axis=1)
d_xy.columns = ['Pred', 'Std', 'Std_err']
d_xy
d_xy.to_csv("./temporal_DLmodels/prediction_std_stderr_probability_lnd3.csv")